In [31]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd

In [32]:
login = 'mankevichpavel@gmail.com'
password = 'googlehf,jnf'
home_page = 'https://websim.worldquantchallenge.com/en/cms/wqc/websim/'
simulate = 'https://websim.worldquantchallenge.com/simulate\n'
logout = 'https://websim.worldquantchallenge.com/simulate/logout'
text = '(vwap / close - vwap + close) * 1'
alpha_stats = ['alpha', 'year', 'long_count', 'short_count', 'pnl', 'sharpe', 'fitness', 
               'returns', 'draw_down', 'turn_over', 'margin']

In [33]:
def driver_relog(driver):
    driver.get('https://websim.worldquantchallenge.com/logout')
    driver.get(home_page)
    log_pass= driver.find_elements_by_class_name('form-control')
    log_pass[0].clear(), log_pass[0].send_keys(login)
    log_pass[1].clear(), log_pass[1].send_keys(password)
    log_pass[1].send_keys(Keys.RETURN)

def stats_error(driver):
    time.sleep(60)
    try:
        stats = driver.find_element_by_id('test-statsBtn') # 2 min wait
        stats.click()
    except NoSuchElementException: 
        driver_relog(driver)
        return False
    else:
        return True
        
def line_error(driver):
    driver.get('https://websim.worldquantchallenge.com/simulate')
    try:
        driver.find_element_by_class_name('CodeMirror-line').click()
    except NoSuchElementException:
        driver_relog(driver)
        return False
    else:
        return True

def download_stats(driver, res_df, i):
    table = driver.find_elements_by_class_name('standard-row')
    for row_id, row in enumerate(table):
        data = row.find_elements_by_tag_name('td')
        res_df.alpha[i * 7 + row_id] = alpha
        res_df.year[i * 7 + row_id] = data[1].text
        res_df.long_count[i * 7 + row_id] = data[3].text
        res_df.short_count[i * 7 + row_id] = data[4].text
        res_df.pnl[i * 7 + row_id] = data[5].text
        res_df.sharpe[i * 7 + row_id] = data[6].text
        res_df.fitness[i * 7 + row_id] = data[7].text
        res_df.returns[i * 7 + row_id] = data[8].text
        res_df.draw_down[i * 7 + row_id] = data[9].text
        res_df.turn_over[i * 7 + row_id] = data[10].text
        res_df.margin[i * 7 + row_id] = data[11].text

In [34]:
driver = webdriver.Chrome(executable_path='C:\webdrivers\chromedriver.exe')  
driver.set_window_size(1366, 768)
driver.implicitly_wait(60)

driver.get(home_page)
log_pass= driver.find_elements_by_class_name('form-control')
log_pass[0].clear(), log_pass[0].send_keys(login)
log_pass[1].clear(), log_pass[1].send_keys(password)
log_pass[1].send_keys(Keys.RETURN)

driver.find_element_by_class_name('caret')

In [35]:
simulate_df = pd.read_csv('alphas_for_simulate.csv')
res_df = pd.DataFrame(index=range(simulate_df.shape[0] * 7), columns=alpha_stats)
save_df = pd.read_csv('simulate_result_rename.csv')

i_start = save_start = save_df.dropna(how='all').shape[0] / 7

In [48]:
login_time = time.time()
line_found = progressbar_found = stats_found = False

while(1):
    try:
        for i in range(simulate_df.shape[0])[i_start:]:
            alpha = simulate_df.iloc[i][0]

            if line_found == False:
                driver.get('https://websim.worldquantchallenge.com/simulate')
                driver.find_element_by_class_name('CodeMirror-line').click()

            action = ActionChains(driver)
            action.send_keys(alpha)
            action.perform()

            simulate = driver.find_elements_by_class_name('col-xs-4')
            simulate[2].click()

            if progressbar_found == False:
                driver.find_element_by_class_name('progressbar-front-text')

            if stats_found == False:
                stats = driver.find_element_by_id('test-statsBtn')
                stats.click()

            download_stats(driver, res_df, i)
            line_found = progressbar_found = stats_found = False
            
            if i % 30 == 0:
                save_df.iloc[save_start * 7:(i + 1) * 7] = res_df.iloc[save_start * 7:(i + 1) * 7]
                save_start = i + 1
                save_df.to_csv('simulate_result_rename.csv', index=False)

            if int(time.time()) - login_time > 10800:
                driver_relog(driver)
                login_time = int(time.time())
            
    except NoSuchElementException as error:
        if 'CodeMirror-line' in error.msg:
            line_found = line_error(driver)
        
        if 'progressbar-front-text' in error.msg:
            print error.msg
    
        if 'test-statsBtn' in error.msg:
            stats = stats_error(driver)
            line_found = progressbar_found = stats
        
        i_start = i 
    
    break
#res_df.to_csv('simulate_result_rename.csv', index=False)


In [45]:
i

In [19]:
driver.implicitly_wait(2)
try:
    stats = driver.find_element_by_id('test-statsBtn')
    stats.click()
    
except NoSuchElementException as error:
    if 'CodeMirror-line' in error.msg:
        line_found = line_error(driver)
        
    if 'progressbar-front-text' in error.msg:
        print error.msg
    
    if 'test-statsBtn' in error.msg:
        stats = stats_error(driver)
        line_found = progressbar_found = stats
        
    i_start = i

no such element: Unable to locate element: {"method":"id","selector":"test-statsBtn"}
  (Session info: chrome=64.0.3282.167)
  (Driver info: chromedriver=2.33.506120 (e3e53437346286c0bc2d2dc9aa4915ba81d9023f),platform=Windows NT 10.0.14393 x86_64)
True


In [ ]:
i

In [ ]:
def stats_error(driver):
    time.sleep(60)
    try:
        stats = driver.find_element_by_id('test-statsBtn') # 2 min wait
        stats.click()
    except NoSuchElementException: 
        driver_relog(driver)
        return False
    else:
        return True
        
def line_error(driver):
    driver.get('https://websim.worldquantchallenge.com/simulate')
    try:
        driver.find_element_by_class_name('CodeMirror-line').click()
    except NoSuchElementException:
        driver_relog(driver)
        return False
    else:
        return True

In [43]:
print l, driver

['asdr', 'as\\sd'] 1
